<a href="https://colab.research.google.com/github/dablro12/data_analyses/blob/main/Part2_Avatar2_Final_project_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# (구글 코랩에서 실습할 경우) 아래 코드를 실행하여 한글 폰트를 설치합니다. 
# 설치가 완료되면, 커널을 재시작합니다. (상단 메뉴에서 [런타임] - [런타임 다시 시작] 을 선택)

# (로컬 PC에서 작업할 경우) 이 코드를 실행하지 않습니다!!!

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20180306-3).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-san

In [4]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [5]:
# 다운로드 받을 폴더를 준비
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [6]:
# 데이터 다운로드 
import torchtext 
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'review.txt'))

14.6MB [00:00, 46.0MB/s]


'/content/data/review.txt'

In [8]:
# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv('./data/review.txt', sep='\t')

# 데이터프레임의 크기
print(data.shape)

# 첫 5행을 출력
data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 2. KoNLPy 한글 형태소 분석

In [12]:
# 코랩에 konlpy 패키지를 설치합니다.
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 46.3 MB/s eta 0:00:00


In [13]:
# 긍정 리뷰를 하나 선택
sample_text = data['document'].iloc[149978]
print(type(sample_text))
print(sample_text)
# 트위터 형태소 분석기(Okt)를 활용
from konlpy.utils import pprint
from konlpy.tag import Okt
okt = Okt()
print(okt)

<class 'str'>
그리 만족스럽진못했어도 7점은 나와야되는것같아 10점줌. 주인공들연기도 훌륭했고 내용도 이정도면 괜찮았다. 해피엔딩으로끝났으면 그저 진부한영화가 되버릴뻔. 그래도 엔딩이 섭섭한건 없지않아있었음. 킬러들의도시도그렇고 콜린파렐을 너무 축축하게 끝내버리는듯


# 3. 텍스트 전처리

In [14]:
# 누락 데이터를 제거
review_data = data['document'].dropna().values

# 학습 속도를 고려하여, 1000개의 샘플을 선택하여 추출
review_data = review_data[:1000]

# 배열의 크기
print(review_data.shape)

# 첫 번째 데이터
print(review_data[0])

(1000,)
아 더빙.. 진짜 짜증나네요 목소리


In [15]:
# 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) >= 3:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[0])

100%|██████████| 1000/1000 [00:13<00:00, 76.30it/s]

1000
목소리


# 4. TF-IDF 벡터로 표현

In [17]:
# 사이킷런 패키지 활용 
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

(1000, 597)
  (0, 152)	1.0


In [18]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서 5개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 5 })

597
{'목소리': 152, '포스터': 546, '교도소': 31, '이야기': 400, '스파이더맨': 260}


In [19]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() } 

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{152: '목소리', 546: '포스터', 31: '교도소', 400: '이야기', 260: '스파이더맨'}


In [20]:
# 첫 번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원 (순서 복원 X)
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[0].indices])
original_text

'목소리'

# Topic Modeling

## 1. LDA

In [23]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(2)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [24]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 2157개의 단어)
print(lda.components_.shape)

(2, 597)


In [25]:
# 2157개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('드라마', 19.52), ('스토리', 18.231), ('연기력', 5.427), ('스릴러', 5.214), ('비디오', 4.595), ('공포영화', 4.479), ('로맨스', 4.468), ('이영화', 4.303), ('화이팅', 3.613), ('최고다', 3.479)]
토픽 유형 2: [('쓰레기', 15.707), ('주인공', 12.721), ('이야기', 10.977), ('마지막', 9.471), ('포스터', 6.358), ('캐스팅', 6.051), ('시청률', 5.058), ('시리즈', 4.941), ('긴장감', 4.456), ('어린이', 4.273)]


## 2. pyLDAvis 시각화

In [26]:
# pyLDAvis 설치
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=68272476797fce87c81eee20925b8980cbe8177de2bef716af59d9b1ea3e2408
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=0f184c9c6846c62ce7d7f6fe6b20e8d8adec84223f6257fdee94c428b8897e54
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [27]:
# LDA 토픽 모델링 결과를 시각화
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


# Sentimental Analysis (감성 분석)

## 1. 머신러닝 알고리즘 활용해 긍정/부정 분류

In [32]:
labels = data['label'].iloc[:1000].values
print(labels.shape)
print(labels[:5])

(1000,)
[0 1 0 0 1]


In [35]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

LogisticRegression()

In [36]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

[0]


## 2. Part 1에서 수집한 리뷰를 긍정, 부정으로 감성 분류 예측

In [39]:
# 판다스 데이터프레임로 읽어오기
test = pd.read_csv('./data/naver_review_Avatar2.csv')

test.head()

,Unnamed: 0,review_uid,review_url,review_title,review_content
0,0,insu****,https://movie.naver.com/movie/bi/mi/reviewread...,"<아바타 2> 주인공 ""네이티리"" 캐릭터 컨셉 아트 이미지 대공개...","아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티븐 랭, 조 샐다나..."
1,1,xnzm****,https://movie.naver.com/movie/bi/mi/reviewread...,2014년에 이글을 읽는사람들에게..,"2014년에 아바타2가 개봉하다니ㅋ\n \n아바타1,, 정말 재밌게 봤었는데\n \..."
2,2,alwa****,https://movie.naver.com/movie/bi/mi/reviewread...,"[필독, 스압] 위대한 지휘관, 쿼리치대령을 추모하며...",영화 아바타는.끝까지 우리 인류의 번영과 이익을 위해 사투를 벌이다 죽어간 위대한 ...
3,3,soqu****,https://movie.naver.com/movie/bi/mi/reviewread...,"아바타2 3,4 제작소식 모음",<< 아바타 속편 제작소식 모음 >>\n \n\n \n \n안녕하세요. 오늘은 ...
4,4,doub****,https://movie.naver.com/movie/bi/mi/reviewread...,아바타2 스토리,전작의 경우 cg는 혁명적이었지만제임스 카메론이 14년동안 구상한거 치곤 스토리가 ...


In [41]:
# 첫 번째 리뷰를 선택
test_sample = test['review_content'][0]
test_sample

'아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티븐 랭, 조 샐다나, 샘 워싱턴\n개봉\n2017 미국\n\n리뷰보기\n\n\n\u200b*** 아바타 2 (2017) ***장르: Sci-Fi, 판타지, 모험, 액션감독: 제임스 카메론주연: 조 셀다나 (네이티리 역), 샘 워싱턴 (제이크 설리 역), 시고니 위버 (그레이스 어거스틴 역), 스티븐 랭 (마일즈 쿼리치 역)...국적: 미국북미 개봉일: 2017년 12월 25일2017년 12월 25일 북미 개봉작으로 기대를 모으고 있는 영화 아바타 2...특히나 주인공 "네이티리" 의 캐릭터 이미지가 공개가 되어서 올려봅니다.배우 조 샐다나와 샘 워싱턴, 시고니 위버 그리고 스티븐 랭까지 전편에서 출연했었던 배우들이 그대로 출연을 하고 있어서더욱 기대가 되는 영화인 것 같습니다. 아무튼 제임스 카메론 감독의 영화 아바타 2편을 시작으로2018년 아바타 3와 2019년 아바타 4까지 진행되고 있어서 앞으로 더욱 기대되는 영화 아바타인 것 같습니다.'

In [42]:
# 한글을 제외하고 전부 제거
test_sample = re.sub(r"[^가-힣]", "", test_sample)
test_sample

'아바타감독제임스카메론출연시고니위버스티븐랭조샐다나샘워싱턴개봉미국리뷰보기아바타장르판타지모험액션감독제임스카메론주연조셀다나네이티리역샘워싱턴제이크설리역시고니위버그레이스어거스틴역스티븐랭마일즈쿼리치역국적미국북미개봉일년월일년월일북미개봉작으로기대를모으고있는영화아바타특히나주인공네이티리의캐릭터이미지가공개가되어서올려봅니다배우조샐다나와샘워싱턴시고니위버그리고스티븐랭까지전편에서출연했었던배우들이그대로출연을하고있어서더욱기대가되는영화인것같습니다아무튼제임스카메론감독의영화아바타편을시작으로년아바타와년아바타까지진행되고있어서앞으로더욱기대되는영화아바타인것같습니다'

In [44]:
# 세글자 이상의 명사만 가지고 오기
tokens = okt.nouns(test_sample)
cleaned_tokens = list()

for word in tokens:
    if len(word) >= 3:
        cleaned_tokens.append(word)
    else:
        pass
cleaned_review = ' '.join(cleaned_tokens)
        
cleaned_review   

'아바타 제임스 카메론 워싱턴 아바타 판타지 제임스 카메론 네이티 리역샘 워싱턴 제이크 레이스 어거스틴 스티븐 개봉일 아바타 네이티 캐릭터 이미지 워싱턴 스티븐 영화인 제임스 카메론 아바타 아바타 아바타 아바타'

In [45]:
# TF-IDF 변환기에 데이터를 입력하여 변환
test_review_tfid = tfid.transform([cleaned_review])

# 배열을 출력
print(test_review_tfid)

  (0, 532)	0.5044706303487678
  (0, 503)	0.4531248037993559
  (0, 391)	0.5345060134487065
  (0, 356)	0.5044706303487678


In [46]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
test_pred = lr.predict(test_review_tfid)[0]
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정"))

분석 결과 부정적인 리뷰로 예측됩니다. 
